# View, annotate, and analyze multi-dimensional images in Python with napari

## 1. A *fast* 2D viewer

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import napari

In [2]:
import tifffile

image = tifffile.imread(
    '/Users/jni/projects/demos/spatialdata-sandbox/'
    'visium_io/data/Visium_Mouse_Olfactory_Bulb_image.tif'
)

In [3]:
image.shape

(10000, 10000, 4)

In [4]:
%matplotlib qt

In [5]:
plt.imshow(image)

In [4]:
viewer, layer = napari.imshow(image)

/Users/jni/conda/envs/all/lib/python3.10/site-packages/plateseg/watershed.py:95: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(cache=True)


## 2. a *multidimensional* viewer

In [7]:
from skimage import data

cells = data.cells3d()

cells.shape

(60, 2, 256, 256)

In [ ]:
data.cells3d?

In [8]:
plt.imshow(cells)

TypeError: Invalid shape (60, 2, 256, 256) for image data

In [9]:
viewer, layer = napari.imshow(
        cells,
        channel_axis=1,
        scale=[0.29, 0.26, 0.26],
        )

## 3. a *lazy* viewer

Tribolium castaneum light sheet microscopy data from the [Cell tracking challenge](http://celltrackingchallenge.net/3d-datasets/) contributed by Akanksha Jain, MPI-CBG Dresden.

In [10]:
import zarr

image = zarr.open('/Users/jni/data/Fluo-N3DL-TRIF/01.ome.zarr/0/')

print(image.nbytes / 1e9)

213.49715712


In [11]:
print(image.shape)

(60, 988, 1868, 964)


In [12]:
print(image.chunks)

(1, 64, 512, 512)


In [14]:
viewer, layer = napari.imshow(image)

In [15]:
layer.contrast_limits

[1.0, 10998.88111888112]

In [16]:
layer.contrast_limits = 1000, 6000

In [17]:
layer.colormap = 'magma'

## 3. a *multiscale* viewer

In [18]:
image1 = zarr.open('/Users/jni/data/Fluo-N3DL-TRIF/01.ome.zarr/1/')
image2 = zarr.open('/Users/jni/data/Fluo-N3DL-TRIF/01.ome.zarr/2/')

del viewer.layers['image']

layer_multi = viewer.add_image(
        [image, image1, image2],
        rendering='attenuated_mip',
        name='tribolium',
        )

... Slides...

## 4. a *layered* viewer

### cryoET demo from napari core developer Alister Burt
https://github.com/alisterburt/napari-cryo-et-demo

In [19]:
import mrcfile
import napari
import numpy as np
import starfile
from scipy.spatial.transform import Rotation as R

# files containing data
tomogram_file = '/Users/jni/data/napari-cryo-et-demo/hiv/01_10.00Apx.mrc'
particles_file = '/Users/jni/data/napari-cryo-et-demo/hiv/01_10.00Apx_particles.star'

# loading data into memory
# tomogram is a numpy array containing image array data
# df is a pandas DataFrame containing table of info from STAR file
with mrcfile.open(tomogram_file) as mrc:
    tomogram = mrc.data.copy()
df = starfile.read(particles_file)

# get particle positions as (n, 3) numpy array from DataFrame
xyz = df[
        ['rlnCoordinateX', 'rlnCoordinateY', 'rlnCoordinateZ']
        ].to_numpy()

# get particle orientations as Euler angles from DataFrame
euler_angles = df[
        ['rlnAngleRot', 'rlnAngleTilt', 'rlnAnglePsi']
        ].to_numpy()

# turn Euler angles into a scipy 'Rotation' object, rotate Z vectors to see
# where they point for the aligned particle
rotations = R.from_euler(
        seq='ZYZ', angles=euler_angles, degrees=True
        ).inv()
rotated_z_vectors = rotations.apply([0, 0, 1])

# set up napari vectors layer data
# (n, 2, 3) array
# dim 0: batch dimension
# dim 1: first row is start point of vector,
#        second is direction vector
# dim 2: components of direction vector e.g. (x, y, z)
zyx = xyz[:, ::-1]
direction_zyx = rotated_z_vectors[:, ::-1]
vectors = np.stack((zyx, direction_zyx), axis=1)

# instantiate a napari viewer
viewer = napari.Viewer()

# add data to viewer
viewer.add_image(
        tomogram,
        blending='translucent_no_depth',
        colormap='gray_r',
        )
viewer.add_points(
        zyx,
        face_color='cornflowerblue',
        size=10,
        )
viewer.add_vectors(
        vectors, length=10, edge_color='orange'
        )
# napari.run()

<Vectors layer 'vectors' at 0x2a3706c50>

### Coral photogrammetry data

Data by Emmanuel Reynaud (University College Dublin) and Luis Gutierrez, at https://doi.org/10.6084/m9.figshare.22348645.  
Demo and implementation by Ashley Anderson (CZI).

`examples/surface_multi_texture.py`

In [20]:
import os
import pooch
from vispy.io import imread, read_mesh

# Download the model
# ------------------
download = pooch.DOIDownloader(progressbar=True)
doi = "10.6084/m9.figshare.22348645.v1"
tmp_dir = pooch.os_cache("napari-surface-texture-example")
os.makedirs(tmp_dir, exist_ok=True)
data_files = {
    "mesh": "PocilloporaDamicornisSkin.obj",
    # "materials": "PocilloporaVerrugosaSkinCrop.mtl",  # not yet supported
    "Texture_0": "PocilloporaDamicornisSkin_Texture_0.jpg",
    "GeneratedMat2": "PocilloporaDamicornisSkin_GeneratedMat2.png",
}
for file_name in data_files.values():
    if not (tmp_dir / file_name).exists():
        download(f"doi:{doi}/{file_name}",
                output_file=tmp_dir / file_name,
                pooch=None,
                )


# Load the model
vertices, faces, _normals, texcoords = read_mesh(
        tmp_dir / data_files["mesh"]
        )

# Load the textures
photo_texture = imread(tmp_dir / data_files["Texture_0"])
generated_texture = imread(tmp_dir / data_files["GeneratedMat2"])

viewer = napari.Viewer(ndisplay=3)

photo_texture_layer = viewer.add_surface(
        (vertices, faces),
        texture=photo_texture,
        texcoords=texcoords,
        name="Texture_0",
        )
generated_texture_layer = viewer.add_surface(
        (vertices, faces),
        texture=generated_texture,
        texcoords=texcoords,
        name="GeneratedMat2",
        )

# deselect layers to avoid performance bug 😬
viewer.layers.selection = {}

## 5. an *annotation* and *proofreading* tool

### interactive segmentation of 3D cells

Semi-automated methods in Python.

In [21]:
viewer, (membrane_layer, nuclei_layer) = napari.imshow(
        cells,
        channel_axis=1,
        name=['membrane', 'nuclei'],
        )

In [22]:
# grab individual channels and convert to float in [0, 1]

membranes = cells[:, 0, :, :] / np.max(cells)
nuclei = cells[:, 1, :, :] / np.max(cells)

In [23]:
from skimage import filters


edges = filters.farid(nuclei)

edges_layer = viewer.add_image(
        edges,
        blending='additive',
        colormap='yellow',
        )

In [24]:
from scipy import ndimage as ndi

denoised = ndi.median_filter(nuclei, size=3)

In [25]:
li_thresholded = denoised > filters.threshold_li(denoised)

threshold_layer = viewer.add_image(
        li_thresholded,
        opacity=0.3,
        )

In [26]:
from skimage import morphology

width = 20

holes_removed = morphology.remove_small_holes(
        li_thresholded, width ** 3
        )

speckle_removed = morphology.remove_small_objects(
        holes_removed, width ** 3
        )

viewer.layers[-1].visible = False

viewer.add_image(
        speckle_removed,
        name='cleaned',
        opacity=0.3,
        );

In [27]:
from skimage import measure

labels = measure.label(speckle_removed)

viewer.layers[-1].visible = False
viewer.add_labels(
        labels,
        opacity=0.5,
        blending='translucent_no_depth'
        )


<Labels layer 'labels' at 0x2a65bd4b0>

In [28]:
# Sean's solution
from scipy import ndimage as ndi
from skimage.feature import peak_local_max

spacing = [0.29, 0.26, 0.26]
distances = ndi.distance_transform_edt(
    speckle_removed, sampling=spacing
)
dt_smoothed = filters.gaussian(distances, sigma=5)
peaks = peak_local_max(dt_smoothed, min_distance=5)

pts_layer = viewer.add_points(
        peaks,
        name="sean's points",
        size=4,
        n_dimensional=True,  # points have 3D "extent"
        )

In [29]:
points_data = pts_layer.data
points_data

array([[ 35.        ,  98.        , 160.        ],
       [ 36.        ,  13.        , 152.        ],
       [ 35.        , 158.        , 110.        ],
       [ 35.        , 219.        ,  84.        ],
       [ 34.        , 184.        ,  49.        ],
       [ 33.        , 237.        , 129.        ],
       [ 34.        ,  71.        , 111.        ],
       [ 35.        , 146.        , 243.        ],
       [ 36.        ,  47.        , 182.        ],
       [ 36.        ,  34.        ,  76.        ],
       [ 35.        ,  11.        ,  21.        ],
       [ 38.        , 140.        ,  38.        ],
       [ 35.        ,  51.        , 231.        ],
       [ 32.        , 202.        , 169.        ],
       [ 35.        , 143.        , 192.        ],
       [ 41.        ,  78.        ,  59.        ],
       [ 41.        ,  98.        ,  49.        ],
       [ 35.        , 103.88632664, 251.68014107],
       [ 35.        , 219.73223006, 250.01329354],
       [ 35.        , 252.23575

In [30]:
from skimage import segmentation, util

markers = util.label_points(points_data, nuclei.shape)
markers_big = morphology.dilation(markers, morphology.ball(5))

segmented = segmentation.watershed(
        edges, markers_big, mask=speckle_removed,
        )

seg_layer = viewer.add_labels(
        segmented, blending='translucent_no_depth',
        )

viewer.layers['labels'].visible = False

... Slides...

## 6. lazy annotation, thank you zarr! 🧊❤️🙏

In [31]:
type(image), image.shape, image.nbytes / 1e9

(zarr.core.Array, (60, 988, 1868, 964), 213.49715712)

In [32]:
viewer = napari.Viewer()
layer_multi = viewer.add_image(
        [image, image1, image2],
        rendering='attenuated_mip',
        name='tribolium',
        )

labels = zarr.open(
        '/Users/jni/data/Fluo-N3DL-TRIF/01-labels.zarr',
        dtype=np.uint32,
        shape=image.shape,
        write_empty_chunks=False,
        chunks=image.chunks,
        )

In [33]:
!ls -a /Users/jni/data/Fluo-N3DL-TRIF/

.                     01_GT                 random.mov
..                    02                    random.png
.DS_Store             02_GT                 random2.png
01                    anim-random.py        random3.png
01-labels.zarr        anim.py               random4.png
01.ome.zarr           backup-anim-random.py tribolium.mov


In [34]:
!ls -a /Users/jni/data/Fluo-N3DL-TRIF/01-labels.zarr

.       ..      .zarray


In [35]:
labels.shape

(60, 988, 1868, 964)

In [36]:
layer = viewer.add_labels(labels)

In [37]:
!ls -a /Users/jni/data/Fluo-N3DL-TRIF/01-labels.zarr

.        ..       .zarray  29.7.1.0


In [38]:
!rm -rf /Users/jni/data/Fluo-N3DL-TRIF/01-labels.zarr

## 7. plays well with others

napariboard

In [39]:
!python /Users/jni/projects/napariboard-proto/napariboard.py

## 8. extensible with plugins

In [40]:
viewer = napari.Viewer()

In [41]:
viewer.open(
        '/Users/jni/data/Fluo-N3DL-TRIF/01.ome.zarr',
        plugin='napari-ome-zarr',
        )

[<Image layer 'tribolium' at 0x2d2dcba90>]

napari-pdf-reader (I shit you not 😂)

In [42]:
!napari ./data/project_jupyter.pdf

/Users/jni/projects/napari/napari/_vispy/layers/image.py:258: UserWarning: data shape (8, 3300, 2550, 3) exceeds GL_MAX_TEXTURE_SIZE 2048 in at least one axis and will be downsampled. Rendering is currently in 3D mode.!


... slides ...